# [운동 동작 분류 AI - Private 8위](https://dacon.io/codeshare/2393)

[catboost](https://dailyheumsi.tistory.com/136) : Category + Boosting     
범주(category)가 많을 때 유용한 알고리즘으로 높은 정확도를 보임.    
학습시간이 다른 모델에 비해 오래 걸리나 예측시간을 매우 빠름.    



[eli5](https://hong-yp-ml-records.tistory.com/51)    
Permutation Importance는 모델을 학습시킨 뒤, 특정 feature의 데이터를 shuffle 했을 때, 검증 데이터 셋에 대한 예측성능을 확인하고 feature importance를 계산한다.



In [15]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)


from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss
from sklearn.model_selection import StratifiedKFold
from sklearn.cluster import KMeans

import eli5
from eli5.sklearn import PermutationImportance

from scipy.stats import skew
from scipy.stats import kurtosis

import math

In [5]:
train = pd.read_csv('../workout/train_features.csv')
test = pd.read_csv('../workout/test_features.csv')
y = pd.read_csv('../workout/train_labels.csv')
submission = pd.read_csv('../workout/sample_submission.csv')

## Custom function 

In [6]:
def range_func(x):
    return np.max(x) - np.min(x)


def iqr_func2(x) :
    q3, q1 = np.percentile(x, [20, 80])
    return q3-q1

def iqr_func3(x) :
    q3, q1 = np.percentile(x, [40, 60])
    return q3-q1

def iqr_func4(x) :
    q3, q1 = np.percentile(x, [15, 95])
    return q3-q1

def premad(x):
    return np.median(np.absolute(x-np.median(x, axis = 0)), axis = 0 )

def preskew(x):
    return skew(x)

def prekurt(x):
    return kurtosis(x, fisher = True ) # 정규 분포 

## Create change Data 

시간 변동에 따른 수치 변화 데이터    

해당 코드는 중복 반복문으로 인해 시간이 다소 소용되기 때문에 Times 간의 변화량을 이후 `groupby().diff()`을 활용하여 따로 처리 

In [ ]:
train_change = pd.DataFrame()
change_train = train.set_index('id').drop('time', axis = 1)

for j in train['id'].unique() :
    temp = change_train[change_train.index == j ]
    
    for i in range(599):
        change = temp.iloc[i+1, : ] - temp.iloc[i,:]
        train_change = train_change.append(change) 

In [ ]:
test_change =  pd.DataFrame()
change_test = test.set_index('id').drop('time', axis = 1)

for j in test['id'].unique():
    temp = change_test[change_test.index == j]
    
    for i in range(599):
        change = temp.iloc[i+1, :] - temp.iloc[i, :]
        test_change = test_change.append(change)

## Feature Engineering 

In [11]:
# 에너지 계산 
train['acc_vector']   = np.sqrt( train['acc_x'] ** 2 + train['acc_y'] ** 2 + train['acc_z'] ** 2 ) 
train['acc_XYvector'] = np.sqrt( train['acc_x'] ** 2 + train['acc_y'] ** 2 )
train['acc_XZvector'] = np.sqrt( train['acc_x'] ** 2 + train['acc_z'] ** 2 )
train['acc_YZvector'] = np.sqrt( train['acc_y'] ** 2 + train['acc_z'] ** 2 ) 

train['gy_vector']    = np.sqrt( train['gy_x'] ** 2 + train['gy_y'] ** 2 + train['gy_z'] ** 2 )
train['gy_XYvector']  = np.sqrt( train['gy_x'] ** 2 + train['gy_y'] ** 2 )
train['gy_XZvector']  = np.sqrt( train['gy_x'] ** 2 + train['gy_z'] ** 2 )
train['gy_YZctor']    = np.sqrt( train['gy_y'] ** 2 + train['gy_z'] ** 2 )

In [12]:
test['acc_vector']   = np.sqrt( test['acc_x'] ** 2 + test['acc_y'] ** 2 + test['acc_z'] ** 2 ) 
test['acc_XYvector'] = np.sqrt( test['acc_x'] ** 2 + test['acc_y'] ** 2 )
test['acc_XZvector'] = np.sqrt( test['acc_x'] ** 2 + test['acc_z'] ** 2 )
test['acc_YZvector'] = np.sqrt( test['acc_y'] ** 2 + test['acc_z'] ** 2 ) 

test['gy_vector']    = np.sqrt( test['gy_x'] ** 2 + test['gy_y'] ** 2 + test['gy_z'] ** 2 )
test['gy_XYvector']  = np.sqrt( test['gy_x'] ** 2 + test['gy_y'] ** 2 )
test['gy_XZvector']  = np.sqrt( test['gy_x'] ** 2 + test['gy_z'] ** 2 )
test['gy_YZctor']    = np.sqrt( test['gy_y'] ** 2 + test['gy_z'] ** 2 )

In [13]:
# 자이로스코프 무게 중심 
train['gy_Centerofgravity'] = (train['gy_x'] + train['gy_y'] + train['gy_z']) / 3
test['gy_Centerofgravity'] = (test['gy_x'] + test['gy_y'] + test['gy_z']) / 3

### [Roll & Pitch](https://now0930.pe.kr/wordpress/?p=1226)


물체가 이동하면서 어떠한 외부요인에 의해 이동하려는 목표 대비 오차(error)가 발생하게 됨.   
이러한 오차를 발생시키는 원인은 yaw, roll, pitch .    
그 중 가속도 센서로 알 수 있는 건 Roll과 Pitch 값.  

<img src = 'RollPitch계산.jpg'> 

In [16]:
pi = math.pi
# Roll & Pitch 
train['roll']          = np.arctan(train['acc_y'] / np.sqrt(train['acc_x']**2 + train['acc_z']**2))
train['pitch']         = np.arctan(train['acc_x'] / np.sqrt(train['acc_y']**2 + train['acc_z']**2))
train['math_roll']     = np.arctan( - train['acc_x'] / np.sqrt(train['acc_y']**2 + train['acc_z']**2)) * (180/pi)
train['math_pitch']    = np.arctan( train['acc_y'] / np.sqrt(train['acc_x']**2 + train['acc_z']**2)) * (180/pi)
train['gy_roll']       = np.arctan(train['gy_y']/np.sqrt(train['gy_x']**2 + train['gy_z']**2))
train['gy_pitch']      = np.arctan(train['gy_x']/np.sqrt(train['gy_y']**2 + train['gy_z']**2))
train['gy_math_roll']  = np.arctan(-train['gy_x'] / np.sqrt(train['gy_y']**2 + train['gy_z']**2)) * (180/pi)
train['gy_math_pitch'] = np.arctan(train['gy_y'] / np.sqrt(train['gy_x']**2 + train['gy_z']**2)) * (180/pi)

In [17]:
test['roll']           = np.arctan(test['acc_y'] / np.sqrt(test['acc_x']**2 + test['acc_z']**2))
test['pitch']          = np.arctan(test['acc_x'] / np.sqrt(test['acc_y']**2 + test['acc_z']**2))
test['math_roll']      = np.arctan( - test['acc_x'] / np.sqrt(test['acc_y']**2 + test['acc_z']**2)) * (180/pi)
test['math_pitch']     = np.arctan( test['acc_y'] / np.sqrt(test['acc_x']**2 + test['acc_z']**2)) * (180/pi)
test['gy_roll']        = np.arctan(test['gy_y']/np.sqrt(test['gy_x']**2 + test['gy_z']**2))
test['gy_pitch']       = np.arctan(test['gy_x']/np.sqrt(test['gy_y']**2 + test['gy_z']**2))
test['gy_math_roll']   = np.arctan(-test['gy_x'] / np.sqrt(test['gy_y']**2 + test['gy_z']**2)) * (180/pi)
test['gy_math_pitch']  = np.arctan(test['gy_y'] / np.sqrt(test['gy_x']**2 + test['gy_z']**2)) * (180/pi)

In [18]:
train.head(3)

,id,time,acc_x,acc_y,acc_z,gy_x,gy_y,gy_z,acc_vector,acc_XYvector,acc_XZvector,acc_YZvector,gy_vector,gy_XYvector,gy_XZvector,gy_YZctor,gy_Centerofgravity,roll,pitch,math_roll,math_pitch,gy_roll,gy_pitch,gy_math_roll,gy_math_pitch
0,0,0,1.206087,-0.179371,-0.148447,-0.591608,-30.549010,-31.676112,1.228355,1.219352,1.215188,0.232832,44.010999,30.554737,31.681636,44.007023,-20.938910,-0.146550,1.380095,-79.073624,-8.396683,-0.767200,-0.013443,0.770209,-43.957305
1,0,1,1.287696,-0.198974,-0.182444,0.303100,-39.139103,-24.927216,1.315689,1.302978,1.300557,0.269956,46.403958,39.140277,24.929059,46.402968,-21.254406,-0.151814,1.364146,-78.159835,-8.698302,-1.003661,0.006532,-0.374246,-57.505519
2,0,2,1.304609,-0.195114,-0.253382,-3.617278,-44.122565,-25.019629,1.343234,1.319119,1.328987,0.319800,50.851423,44.270593,25.279765,50.722604,-24.253157,-0.145773,1.330405,-76.226605,-8.352177,-1.050508,-0.071194,4.079137,-60.189701


In [23]:
train.columns[:17]

Index(['id', 'time', 'acc_x', 'acc_y', 'acc_z', 'gy_x', 'gy_y', 'gy_z',
       'acc_vector', 'acc_XYvector', 'acc_XZvector', 'acc_YZvector',
       'gy_vector', 'gy_XYvector', 'gy_XZvector', 'gy_YZctor',
       'gy_Centerofgravity'],
      dtype='object')

In [40]:
temp_train = train.iloc[:,:17].drop('time', axis = 1).groupby('id').agg(['max','min','mean','std','median'])

In [41]:
train_processing = train.drop('time', axis = 1).groupby('id').agg([range_func, iqr_func2, iqr_func3, iqr_func4,premad, preskew, prekurt])

In [151]:
train_process = pd.concat([temp_train, train_processing], axis = 1)

In [152]:
temp_test = test.iloc[:,:17].drop('time',axis = 1).groupby('id').agg(['max','min','mean','std','median'])
test_processing = test.drop('time', axis = 1).groupby('id').agg([range_func, iqr_func2, iqr_func3, iqr_func4, premad, preskew, prekurt])

In [153]:
test_process = pd.concat([temp_test, test_processing], axis = 1)
print(train_process.shape, test_process.shape)

(3125, 236) (782, 236)


In [154]:
train_process.columns = [i[0] +'_'+ i[1] for i in train_process.columns]
test_process.columns = [i[0] + '_'+ i[1] for i in test_process.columns]

In [155]:
train_process.columns

Index(['acc_x_max', 'acc_x_min', 'acc_x_mean', 'acc_x_std', 'acc_x_median',
       'acc_y_max', 'acc_y_min', 'acc_y_mean', 'acc_y_std', 'acc_y_median',
       ...
       'gy_math_roll_premad', 'gy_math_roll_preskew', 'gy_math_roll_prekurt',
       'gy_math_pitch_range_func', 'gy_math_pitch_iqr_func2',
       'gy_math_pitch_iqr_func3', 'gy_math_pitch_iqr_func4',
       'gy_math_pitch_premad', 'gy_math_pitch_preskew',
       'gy_math_pitch_prekurt'],
      dtype='object', length=236)

In [156]:
# 표준 편차 평균 
train_process['acc_std_mean'] = ( train_process['acc_x_std'] + train_process['acc_y_std'] + train_process['acc_z_std']) /3
train_process['gy_std_mean'] = ( train_process['gy_x_std'] + train_process['gy_y_std'] + train_process['gy_z_std']) /3


test_process['acc_std_mean'] = ( test_process['acc_x_std'] + test_process['acc_y_std'] + test_process['acc_z_std']) /3
test_process['gy_std_mean'] = ( test_process['gy_x_std'] + test_process['gy_y_std'] + test_process['gy_z_std']) /3


In [157]:
# 가속도계 첫번째 데이터 

train_acc_h1 = train.iloc[:,:5].drop_duplicates(['id'], keep ='first').drop('time', axis = 1)
train_acc_h1.columns = ['id','first_acc_x','first_acc_y','first_acc_z']
train_acc_h1.set_index('id', inplace =True)
train_acc_h1.head(2)

,first_acc_x,first_acc_y,first_acc_z
id,,,
0,1.206087,-0.179371,-0.148447
1,-0.211795,-0.078760,0.854627


In [158]:
test_acc_h1 = test.iloc[:,:5].drop_duplicates(['id'], keep ='first').drop('time', axis = 1)
test_acc_h1.columns = ['id','first_acc_x','first_acc_y','first_acc_z']
test_acc_h1.set_index('id', inplace =True)
test_acc_h1.head(2)

,first_acc_x,first_acc_y,first_acc_z
id,,,
3125,-0.628100,-0.160155,0.151487
3126,0.304222,1.529324,-0.338265


In [159]:
# 가속도계 첫 3초

train.iloc[:,:5][train['time']<150] # 0.02초 * 150 Times = 3초 

,id,time,acc_x,acc_y,acc_z
0,0,0,1.206087,-0.179371,-0.148447
1,0,1,1.287696,-0.198974,-0.182444
2,0,2,1.304609,-0.195114,-0.253382
3,0,3,1.293095,-0.230366,-0.215210
4,0,4,1.300887,-0.187757,-0.222523
...,...,...,...,...,...
1874545,3124,145,-0.807254,-0.555749,0.263630
1874546,3124,146,-0.773207,-0.591142,0.283675
1874547,3124,147,-0.747712,-0.602606,0.305241
1874548,3124,148,-0.715429,-0.609984,0.338578


In [160]:
train_acc_3 = train.iloc[:,:5][train['time']<150].drop('time',axis = 1).groupby('id').mean()  
train_acc_3.columns = ['head_acc_x','head_acc_y','head_acc_z']
train_acc_3.head(3)

,head_acc_x,head_acc_y,head_acc_z
id,,,
0,0.930401,-0.204424,-0.357682
1,-0.313634,-0.567014,0.471395
2,0.005221,-0.113100,0.600223


In [161]:
test_acc_3 = test.iloc[:,:5][test['time']<150].drop('time',axis = 1).groupby('id').mean()  
test_acc_3.columns = ['head_acc_x','head_acc_y','head_acc_z']
test_acc_3.head(3)

,head_acc_x,head_acc_y,head_acc_z
id,,,
3125,-0.959972,-0.027414,-0.263373
3126,-0.455545,0.862888,-0.468532
3127,0.469283,0.979939,-0.211281


In [162]:
train_preprocess = pd.concat([train_process, train_acc_h1, train_acc_3], axis= 1)
test_preprocess = pd.concat([test_process, test_acc_h1, test_acc_3], axis = 1)

In [163]:
print(train_preprocess.shape, test_preprocess.shape)

(3125, 244) (782, 244)


In [164]:
# 자이로스코프 첫 3초 

gy = ['id', 'gy_x','gy_y','gy_z']
train_gy_3 = train[gy][train['time'] < 150].groupby('id').mean()
train_gy_3.columns = ['head_gy_x','head_gy_y','head_gy_z']

In [165]:
test_gy_3 = test[gy][test['time'] <150].groupby('id').mean()
test_gy_3.columns = ['head_gy_x','head_gy_y','head_gy_z']

In [166]:
train_preprocess = pd.concat([train_preprocess, train_gy_3], axis = 1)
test_preprocess = pd.concat([test_preprocess, test_gy_3], axis = 1)

In [167]:
print(train_preprocess.shape, test_preprocess.shape)

(3125, 247) (782, 247)


In [168]:
# 센서 변화량 측정  
train.groupby(['id'])['acc_x', 'acc_y','acc_z','gy_x','gy_y','gy_z'].diff() 

,acc_x,acc_y,acc_z,gy_x,gy_y,gy_z
0,NaN,NaN,NaN,NaN,NaN,NaN
1,0.081610,-0.019602,-0.033997,0.894708,-8.590094,6.748896
2,0.016913,0.003859,-0.070939,-3.920378,-4.983461,-0.092413
3,-0.011514,-0.035252,0.038173,6.330264,-9.475278,-2.434384
4,0.007792,0.042609,-0.007313,1.573721,-4.308718,-0.507222
...,...,...,...,...,...,...
1874995,0.029693,0.006073,0.016001,-3.012893,-0.692606,-0.167602
1874996,0.029493,-0.000109,0.035515,-0.781232,2.216855,-0.334650
1874997,0.018306,-0.008159,0.034891,2.275994,3.020737,-2.740038
1874998,0.034197,-0.015940,0.009582,4.236545,-0.363423,-0.894353


In [169]:
temp = pd.concat([train['id'], train.groupby(['id'])['acc_x', 'acc_y','acc_z','gy_x','gy_y','gy_z'].diff()], axis= 1 ).dropna()

In [170]:
temp['Croll'] = np.arctan(temp['acc_y']/ np.sqrt(temp['acc_x']**2 + temp['acc_z']**2))
temp['Cpitch'] = np.arctan(temp['acc_x']/np.sqrt(temp['acc_y']**2 + temp['acc_z']**2))
temp['Cmath_roll'] = np.arctan(-temp['acc_x']/ np.sqrt(temp['acc_y']**2 + temp['acc_z']**2)) * (180/pi)
temp['Cmath_pitch'] = np.arctan(temp['acc_y']/ np.sqrt(temp['acc_x']**2 + temp['acc_z']**2)) * (180/pi)

temp['Cgyroll'] = np.arctan(temp['gy_y']/np.sqrt(temp['gy_x']**2 + temp['gy_z']**2))
temp['Cgypitch'] = np.arctan(temp['gy_x']/np.sqrt(temp['gy_y']**2 + temp['gy_z']**2))
temp['Cgymath_roll'] = np.arctan(-temp['gy_x']/np.sqrt(temp['gy_y']**2 + temp['gy_z']**2)) *(180/pi)
temp['Cgymath_pitch'] = np.arctan(temp['gy_y']/np.sqrt(temp['gy_x']**2 + temp['gy_z']**2)) *(180/pi)

In [171]:
ca_list = ['id','Cacc_x','Cacc_y','Cacc_z','Cgy_x','Cgy_y','Cgy_z']
temp.columns = ca_list + list(temp.iloc[:,7:].columns)

In [172]:
tmp = temp.groupby('id').agg([range_func, iqr_func2, iqr_func3, iqr_func4, premad, preskew, prekurt])
tmp.head(3)

Cacc_x                                                               \
   range_func iqr_func2 iqr_func3 iqr_func4    premad   preskew    prekurt   
id                                                                           
0    0.350553 -0.086247 -0.027968 -0.140360  0.034023  0.201069   0.184567   
1    1.834542 -0.116587 -0.021564 -0.259661  0.038897  0.309264  15.576594   
2    1.125718 -0.124280 -0.028638 -0.323444  0.043256 -0.002835   3.275095   

       Cacc_y                                                              \
   range_func iqr_func2 iqr_func3 iqr_func4    premad   preskew   prekurt   
id                                                                          
0    0.510419 -0.108800 -0.028574 -0.180126  0.041689 -0.224045  0.799730   
1    0.804631 -0.111154 -0.023733 -0.223176  0.040081  0.127286  3.494794   
2    1.907342 -0.141848 -0.029962 -0.394710  0.051895 -0.053779  5.570731   

       Cacc_z                                                              \
   range_func iqr_func2 iqr_func3 iqr_func4    premad   preskew   prekurt   
id                                                                          
0    0.500255 -0.105577 -0.028014 -0.188058  0.038816  0.166563  1.211711   
1    0.851648 -0.085792 -0.020637 -0.186626  0.033583  0.902761  5.991293   
2    0.952028 -0.110865 -0.031669 -0.204149  0.041540  0.075537  4.722602   

         Cgy_x                                                                \
    range_func  iqr_func2 iqr_func3  iqr_func4    premad   preskew   prekurt   
id                                                                             
0    65.480260 -13.889571 -3.869127 -23.585541  5.419533  0.041505  0.868294   
1   273.314699 -21.121511 -4.788293 -60.485939  7.305740  0.256503  5.516639   
2   221.741820 -19.978814 -4.692393 -52.271026  7.473219  0.205375  4.997718   

         Cgy_y                                                                \
    range_func  iqr_func2 iqr_func3  iqr_func4    premad   preskew   prekurt   
id                                                                             
0    37.129643  -8.924823 -2.690859 -14.098464  3.699058  0.359877  0.502568   
1   254.730500 -18.228214 -3.362948 -40.859910  6.293965  0.320409  9.681797   
2   130.866485 -24.397771 -7.384581 -43.581835  9.835206  0.172387  1.098132   

         Cgy_z                                                                \
    range_func  iqr_func2 iqr_func3  iqr_func4    premad   preskew   prekurt   
id                                                                             
0    34.134910  -7.172572 -2.040204 -12.030779  2.833154  0.200250  0.693328   
1   131.191643 -11.465256 -2.487675 -33.794652  3.947651  0.068497  3.872017   
2    63.751136  -9.539324 -2.311907 -19.274785  3.842362  0.351238  2.389410   

        Croll                                                              \
   range_func iqr_func2 iqr_func3 iqr_func4    premad   preskew   prekurt   
id                                                                          
0    3.034129 -1.458924 -0.420265 -2.101978  0.598439 -0.004368 -0.934040   
1    2.949186 -1.355521 -0.467312 -1.940425  0.577064 -0.012845 -0.945990   
2    3.006554 -1.518542 -0.411151 -2.178463  0.621968 -0.003137 -0.991472   

       Cpitch                                                              \
   range_func iqr_func2 iqr_func3 iqr_func4    premad   preskew   prekurt   
id                                                                          
0    2.871435 -1.060137 -0.331797 -1.652963  0.415447  0.051005 -0.513209   
1    3.069593 -1.448206 -0.439248 -2.033844  0.578359  0.000440 -0.955137   
2    2.991025 -1.265548 -0.327568 -1.908854  0.516071 -0.009967 -0.731086   

    Cmath_roll                                                         \
    range_func  iqr_func2  iqr_func3   iqr_func4     premad   preskew   
id                                                                      
0   164.521122 -60.74139

In [173]:
train_preprocess = pd.concat([train_preprocess, tmp], axis = 1)

In [174]:
temp = pd.concat([test['id'], test.groupby(['id'])['acc_x', 'acc_y','acc_z','gy_x','gy_y','gy_z'].diff()], axis= 1 ).dropna()
temp['Croll'] = np.arctan(temp['acc_y']/ np.sqrt(temp['acc_x']**2 + temp['acc_z']**2))
temp['Cpitch'] = np.arctan(temp['acc_x']/np.sqrt(temp['acc_y']**2 + temp['acc_z']**2))
temp['Cmath_roll'] = np.arctan(-temp['acc_x']/ np.sqrt(temp['acc_y']**2 + temp['acc_z']**2)) * (180/pi)
temp['Cmath_pitch'] = np.arctan(temp['acc_y']/ np.sqrt(temp['acc_x']**2 + temp['acc_z']**2)) * (180/pi)

temp['Cgyroll'] = np.arctan(temp['gy_y']/np.sqrt(temp['gy_x']**2 + temp['gy_z']**2))
temp['Cgypitch'] = np.arctan(temp['gy_x']/np.sqrt(temp['gy_y']**2 + temp['gy_z']**2))
temp['Cgymath_roll'] = np.arctan(-temp['gy_x']/np.sqrt(temp['gy_y']**2 + temp['gy_z']**2)) *(180/pi)
temp['Cgymath_pitch'] = np.arctan(temp['gy_y']/np.sqrt(temp['gy_x']**2 + temp['gy_z']**2)) *(180/pi)

temp.columns = ca_list + list(temp.iloc[:,7:].columns)
tmp = temp.groupby('id').agg([range_func, iqr_func2, iqr_func3, iqr_func4, premad, preskew, prekurt])
test_preprocess = pd.concat([test_preprocess, tmp], axis = 1)

In [175]:
print(train_preprocess.shape, test_preprocess.shape)

(3125, 345) (782, 345)


## Clustering Kmeans
5개의 군집을 새로운 파생변수로 사용. 

In [176]:
model = KMeans(n_clusters = 5, random_state = 20)
model.fit(train_preprocess)
train_predict = model.predict(train_preprocess)

In [177]:
train_preprocess['cluster'] = train_predict

In [178]:
test_predict = model.predict(test_preprocess)
test_preprocess['cluster'] = test_predict

In [179]:
train_preprocess.iloc[:,247:-1].columns

Index([       ('Cacc_x', 'range_func'),         ('Cacc_x', 'iqr_func2'),
               ('Cacc_x', 'iqr_func3'),         ('Cacc_x', 'iqr_func4'),
                  ('Cacc_x', 'premad'),           ('Cacc_x', 'preskew'),
                 ('Cacc_x', 'prekurt'),        ('Cacc_y', 'range_func'),
               ('Cacc_y', 'iqr_func2'),         ('Cacc_y', 'iqr_func3'),
               ('Cacc_y', 'iqr_func4'),            ('Cacc_y', 'premad'),
                 ('Cacc_y', 'preskew'),           ('Cacc_y', 'prekurt'),
              ('Cacc_z', 'range_func'),         ('Cacc_z', 'iqr_func2'),
               ('Cacc_z', 'iqr_func3'),         ('Cacc_z', 'iqr_func4'),
                  ('Cacc_z', 'premad'),           ('Cacc_z', 'preskew'),
                 ('Cacc_z', 'prekurt'),         ('Cgy_x', 'range_func'),
                ('Cgy_x', 'iqr_func2'),          ('Cgy_x', 'iqr_func3'),
                ('Cgy_x', 'iqr_func4'),             ('Cgy_x', 'premad'),
                  ('Cgy_x', 'preskew'),            

In [180]:
train_preprocess.iloc[:,-1:].columns

Index(['cluster'], dtype='object')

In [181]:
columns_names = list(train_preprocess.iloc[:,:247].columns)
columns_names.extend([ i[0]+'_'+i[1] for i in train_preprocess.iloc[:,247:-1].columns ] )
columns_names.extend(list(train_preprocess.iloc[:,-1:].columns))

In [182]:
len(columns_names)

346

In [183]:
train_preprocess.columns = columns_names
test_preprocess.columns = columns_names

In [184]:
train_preprocess.head(3)

,acc_x_max,acc_x_min,acc_x_mean,acc_x_std,acc_x_median,acc_y_max,acc_y_min,acc_y_mean,acc_y_std,acc_y_median,acc_z_max,acc_z_min,acc_z_mean,acc_z_std,acc_z_median,gy_x_max,gy_x_min,gy_x_mean,gy_x_std,gy_x_median,gy_y_max,gy_y_min,gy_y_mean,gy_y_std,gy_y_median,gy_z_max,gy_z_min,gy_z_mean,gy_z_std,gy_z_median,acc_vector_max,acc_vector_min,acc_vector_mean,acc_vector_std,acc_vector_median,acc_XYvector_max,acc_XYvector_min,acc_XYvector_mean,acc_XYvector_std,acc_XYvector_median,acc_XZvector_max,acc_XZvector_min,acc_XZvector_mean,acc_XZvector_std,acc_XZvector_median,acc_YZvector_max,acc_YZvector_min,acc_YZvector_mean,acc_YZvector_std,acc_YZvector_median,gy_vector_max,gy_vector_min,gy_vector_mean,gy_vector_std,gy_vector_median,gy_XYvector_max,gy_XYvector_min,gy_XYvector_mean,gy_XYvector_std,gy_XYvector_median,gy_XZvector_max,gy_XZvector_min,gy_XZvector_mean,gy_XZvector_std,gy_XZvector_median,gy_YZctor_max,gy_YZctor_min,gy_YZctor_mean,gy_YZctor_std,gy_YZctor_median,gy_Centerofgravity_max,gy_Centerofgravity_min,gy_Centerofgravity_mean,gy_Centerofgravity_std,gy_Centerofgravity_median,acc_x_range_func,acc_x_iqr_func2,acc_x_iqr_func3,acc_x_iqr_func4,acc_x_premad,acc_x_preskew,acc_x_prekurt,acc_y_range_func,acc_y_iqr_func2,acc_y_iqr_func3,acc_y_iqr_func4,acc_y_premad,acc_y_preskew,acc_y_prekurt,acc_z_range_func,acc_z_iqr_func2,acc_z_iqr_func3,acc_z_iqr_func4,acc_z_premad,acc_z_preskew,acc_z_prekurt,gy_x_range_func,gy_x_iqr_func2,gy_x_iqr_func3,gy_x_iqr_func4,gy_x_premad,gy_x_preskew,gy_x_prekurt,gy_y_range_func,gy_y_iqr_func2,gy_y_iqr_func3,gy_y_iqr_func4,gy_y_premad,gy_y_preskew,gy_y_prekurt,gy_z_range_func,gy_z_iqr_func2,gy_z_iqr_func3,gy_z_iqr_func4,gy_z_premad,gy_z_preskew,gy_z_prekurt,acc_vector_range_func,acc_vector_iqr_func2,acc_vector_iqr_func3,acc_vector_iqr_func4,acc_vector_premad,acc_vector_preskew,acc_vector_prekurt,acc_XYvector_range_func,acc_XYvector_iqr_func2,acc_XYvector_iqr_func3,acc_XYvector_iqr_func4,acc_XYvector_premad,acc_XYvector_preskew,acc_XYvector_prekurt,acc_XZvector_range_func,acc_XZvector_iqr_func2,acc_XZvector_iqr_func3,acc_XZvector_iqr_func4,acc_XZvector_premad,acc_XZvector_preskew,acc_XZvector_prekurt,acc_YZvector_range_func,acc_YZvector_iqr_func2,acc_YZvector_iqr_func3,acc_YZvector_iqr_func4,acc_YZvector_premad,acc_YZvector_preskew,acc_YZvector_prekurt,gy_vector_range_func,gy_vector_iqr_func2,gy_vector_iqr_func3,gy_vector_iqr_func4,gy_vector_premad,gy_vector_preskew,gy_vector_prekurt,gy_XYvector_range_func,gy_XYvector_iqr_func2,gy_XYvector_iqr_func3,gy_XYvector_iqr_func4,gy_XYvector_premad,gy_XYvector_preskew,gy_XYvector_prekurt,gy_XZvector_range_func,gy_XZvector_iqr_func2,gy_XZvector_iqr_func3,gy_XZvector_iqr_func4,gy_XZvector_premad,gy_XZvector_preskew,gy_XZvector_prekurt,gy_YZctor_range_func,gy_YZctor_iqr_func2,gy_YZctor_iqr_func3,gy_YZctor_iqr_func4,gy_YZctor_premad,gy_YZctor_preskew,gy_YZctor_prekurt,gy_Centerofgravity_range_func,gy_Centerofgravity_iqr_func2,gy_Centerofgravity_iqr_func3,gy_Centerofgravity_iqr_func4,gy_Centerofgravity_premad,gy_Centerofgravity_preskew,gy_Centerofgravity_prekurt,roll_range_func,roll_iqr_func2,roll_iqr_func3,roll_iqr_func4,roll_premad,roll_preskew,roll_prekurt,pitch_range_func,pitch_iqr_func2,pitch_iqr_func3,pitch_iqr_func4,pitch_premad,pitch_preskew,pitch_prekurt,math_roll_range_func,math_roll_iqr_func2,math_roll_iqr_func3,math_roll_iqr_func4,math_roll_premad,math_roll_preskew,math_roll_prekurt,math_pitch_range_func,math_pitch_iqr_func2,math_pitch_iqr_func3,math_pitch_iqr_func4,math_pitch_premad,math_pitch_preskew,math_pitch_prekurt,gy_roll_range_func,gy_roll_iqr_func2,gy_roll_iqr_func3,gy_roll_iqr_func4,gy_roll_premad,gy_roll_preskew,gy_roll_prekurt,gy_pitch_range_func,gy_pitch_iqr_func2,gy_pitch_iqr_func3,gy_pitch_iqr_func4,gy_pitch_premad,gy_pitch_preskew,gy_pitch_prekurt,gy_math_roll_range_func,gy_math_roll_iqr_func2,gy_math_roll_iqr_func3,gy_math_roll_iqr_func4,gy_math_roll_premad,gy_math_roll_preskew,gy_math_roll_prekurt,gy_math_pitch_range_func,gy_math_pitch_iqr_fu

In [185]:
train_preprocess.columns.nunique()

346

In [187]:
drop_columns = ['gy_roll_range_func','gy_XZvector_iqr_func3','gy_XZvector_median','gy_Centerofgravity_prekurt']
train_preprocess[drop_columns]

,gy_roll_range_func,gy_XZvector_iqr_func3,gy_XZvector_median,gy_Centerofgravity_prekurt
id,,,,
0,2.833836,-7.117599,21.917642,-0.204728
1,3.082769,-35.747911,64.964235,5.025850
2,3.116418,-6.973994,19.663983,-0.675779
3,2.849506,-11.490834,31.247075,3.489938
4,2.853791,-15.268916,15.534423,19.205443
...,...,...,...,...
3120,2.806288,-11.746323,11.602243,9.594686
3121,2.847665,-10.352061,13.284098,2.198706
3122,2.839952,-60.795614,175.134085,-0.990394


## Catboost Model 
depth = [3,4,5]

In [188]:
y = y['label']
train_df = train_preprocess.drop(drop_columns, axis = 1) 

In [193]:
# depth = 3
cat_depth3 = CatBoostClassifier(random_state = 20,
                               iterations= 26000, task_type = 'GPU',
                               depth = 3, bootstrap_type ='Bernoulli',
                               learning_rate = 0.007932)

# depth = 4 
cat_depth4 = CatBoostClassifier(random_state = 20,
                               iterations= 22000, task_type = 'GPU',
                               depth = 4, bootstrap_type ='Bernoulli',
                               learning_rate = 0.007932)

# depth = 5 
cat_depth5 = CatBoostClassifier(random_state = 20,
                               iterations= 13000, task_type = 'GPU',
                               depth = 5, bootstrap_type ='Bernoulli',
                               learning_rate = 0.007932)

In [192]:
import time 
import datetime 

In [ ]:
start = time.time()
cat_depth3.fit(train_df, y)
cat_depth4.fit(train_df, y)
cat_depth5.fit(train_df, y)

sec = time.time()- start

In [ ]:
print('3개 모델 학습 소요시간 : ', str(datetime.timedelta(seconds = sec)).split('.'))

In [ ]:
test_df = test_preprocess.drop(drop_columns, axis =1 )

predict_depth3 = cat_depth3.predict_proba(test_df)
predict_depth4 = cat_depth4.predict_proba(test_df)
predict_depth5 = cat_depth5.predict_proba(test_df)

#### Soft Voting 

In [ ]:
predict = predict_depth3 * 0.45 + predict_depth4 * 0.35 + predict_depth5 * 0.2